In [187]:
import numpy as np
import pandas as pd 
import datetime 
from string import punctuation
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import spacy
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn import metrics
import re 
import matplotlib.pyplot as plt

In [188]:
import advertools as adv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [189]:
classified = pd.read_excel('tweet_class.xlsx')

In [190]:
classified = classified[classified['label'].isna()== False]

In [191]:
classified.head()

,Unnamed: 0,Tweet Id,Text,label
0,1,1421621483521158912,شدخل كويلي بعدين ياقلبي انت موافق يدخل بجسم...,neg
1,4,1421618720405863936,سالت دكتوره تقول عادي فيه حوامل اخذو لقاح,pos
2,5,1421618331946266880,جتني كررونا قبل والان اخذت لقاح وصرت محصنه اك...,neg
3,6,1421618142556611072,معليش والله اخذتي اللقاح ولا ما اخذتيه ما راح...,pos
4,7,1421618038810497024,رسمياًوزارة التعليم على جميع منسوبي التعليم ال...,neu


In [192]:
classified['label'].value_counts()

neu    241
neg    137
pos    119
Name: label, dtype: int64

In [193]:
classified.shape

(497, 4)

In [194]:
classified.drop(['Unnamed: 0', 'Tweet Id'], axis= 1, inplace=True)

In [195]:
classified['Text'][12]

'لاللتطعيمالاجباري مع اخر يوم هذه الزبدة لدي الانظمة الصحية من الاجبار لا يهم تموت او تتعافي المهم هو ان لا ترهقنا عندما تصاب وتستنزف مقدراتنا الصحية ونتكلف عليك الاف الدولارات نعطيك لقاح ب دولار يخفف اعراض الاصابه وانت وحظك اما تموت او تحيا عليلا'

In [196]:
# drop punctuation 
classified['Text']= classified['Text'].str.replace('[~,,)'',|,RT,/,(,\n,_,!,:,-,1,2,؟,3,4,5,6,7,8,9,0,&,;,#,?,.,="]', '')
classified['Text']

0         شدخل كويلي بعدين ياقلبي انت موافق يدخل بجسم...
1              سالت دكتوره تقول عادي فيه حوامل اخذو لقاح
2       جتني كررونا قبل والان اخذت لقاح وصرت محصنه اك...
3       معليش والله اخذتي اللقاح ولا ما اخذتيه ما راح...
4      رسمياًوزارة التعليم على جميع منسوبي التعليم ال...
                             ...                        
493        هو ماغلط هو كذب ، جالس يقول ان كـورونا انص...
494      لا مو من حقه واذا كان في كرونا و اعدى الباقي...
495    حملة تطعيم لقاح كورونا بدار الملاحظة الاجتماعي...
496    وقالت الدراسة، التي تم إجراؤها خلال الفترة الو...
497    وأوصى مسؤولو الصحة في إسرائيل، بإعطاء جرعات مع...
Name: Text, Length: 497, dtype: object

In [197]:
classified['Text'] = classified['Text'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

In [198]:
stop_words= list(stopwords.words('arabic'))
print(stop_words)

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [199]:
new_stopwords= ["والله","مساء الخير","يعني","والحين","عنه","انت","طيب" ,"الي"]
stop_words = stop_words + new_stopwords

In [200]:
classified['Text_tokz'] = [word_tokenize(w) for w in classified['Text']]

In [201]:
classified['without_sw']= classified['Text_tokz'].apply(lambda x: [item for item in x if item not in stop_words])

In [202]:
stemmer = nltk.ISRIStemmer()
classified['stem_w']= classified['without_sw'].apply(lambda x: [stemmer.stem(w) for w in x])

In [203]:
lemmatizer = nltk.WordNetLemmatizer()
classified['lemma_w']= classified['without_sw'].apply(lambda x: [lemmatizer.lemmatize(w) for w in x])

In [204]:
classified['clean_data'] = classified['stem_w'].apply(' '.join)

In [205]:
classified.sample(5)

,Text,label,Text_tokz,without_sw,stem_w,lemma_w,clean_data
146,أبلغت هيئة الطيران المدني السعودي الشركات العا...,neu,"[أبلغت, هيئة, الطيران, المدني, السعودي, الشركا...","[أبلغت, هيئة, الطيران, المدني, السعودي, الشركا...","[بلغ, هيئ, طير, دني, سعد, شرك, عمل, قطع, سمح, ...","[أبلغت, هيئة, الطيران, المدني, السعودي, الشركا...",بلغ هيئ طير دني سعد شرك عمل قطع سمح نقل سيح حص...
456,مدري ليه غير توصيته ايواااااا لانهم عرفوا ان ...,neg,"[مدري, ليه, غير, توصيته, ايواااااا, لانهم, عرف...","[مدري, ليه, توصيته, ايواااااا, لانهم, عرفوا, ا...","[دري, ليه, وصت, ايواااااا, لان, عرف, ان, لقح, ...","[مدري, ليه, توصيته, ايواااااا, لانهم, عرفوا, ا...",دري ليه وصت ايواااااا لان عرف ان لقح فشل نسب ا...
323,انا تعبانه وعندي أمراض مزمنه واخذ دوا سيوله ...,neg,"[انا, تعبانه, وعندي, أمراض, مزمنه, واخذ, دوا, ...","[انا, تعبانه, وعندي, أمراض, مزمنه, واخذ, دوا, ...","[انا, تعب, عند, مرض, زمن, وخذ, دوا, سيل, اخذ, ...","[انا, تعبانه, وعندي, أمراض, مزمنه, واخذ, دوا, ...",انا تعب عند مرض زمن وخذ دوا سيل اخذ لقح سبب جل...
96,منجد البندول للحامل بس لضروره القصوى تاخذه ...,neg,"[منجد, البندول, للحامل, بس, لضروره, القصوى, تا...","[منجد, البندول, للحامل, لضروره, القصوى, تاخذه,...","[نجد, ندل, حمل, ضرر, قصى, تخذ, ماتكثر, وهم, عط...","[منجد, البندول, للحامل, لضروره, القصوى, تاخذه,...",نجد ندل حمل ضرر قصى تخذ ماتكثر وهم عطو لقح ضرر...
103,عندي استفسار اخدت جرعه من لقاح كورونا وما اتس...,neu,"[عندي, استفسار, اخدت, جرعه, من, لقاح, كورونا, ...","[عندي, استفسار, اخدت, جرعه, لقاح, كورونا, اتسج...","[عند, فسر, اخد, جرع, لقح, كورو, سجل, حين]","[عندي, استفسار, اخدت, جرعه, لقاح, كورونا, اتسج...",عند فسر اخد جرع لقح كورو سجل حين


In [206]:
X= classified['Text']
y= classified['label']

In [207]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

print('Training Data Shape:', X_train.shape)
print('Testing Data Shape: ', X_test.shape)

Training Data Shape: (347,)
Testing Data Shape:  (150,)


In [208]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

In [209]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


clf = LinearSVC()
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC(C=1))])

text_clf.fit(X_train, y_train)
predictions = text_clf.predict(X_test)



In [210]:
print ("\nConfusion matrix:")
cf_metric = metrics.confusion_matrix(y_test,predictions)
print (cf_metric)
                                     
print ("\nClassification report:")
c_report = metrics.classification_report(y_test,predictions)
print (c_report)
                                     
acc = metrics.accuracy_score(y_test,predictions)      
print ("\nAccuracy score=",acc)


Confusion matrix:
[[30  9  6]
 [ 6 74  3]
 [14 13 10]]

Classification report:
              precision    recall  f1-score   support

         neg       0.60      0.67      0.63        45
         neu       0.77      0.89      0.83        83
         pos       0.53      0.27      0.36        37

    accuracy                           0.69       165
   macro avg       0.63      0.61      0.61       165
weighted avg       0.67      0.69      0.67       165


Accuracy score= 0.6909090909090909


In [211]:
'''
# splitting into train and tests
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size =.2, random_state=100)

# make pipeline
clf =  LogisticRegression()
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LogisticRegression(solver='lbfgs'))])

# create and fit the model

model.fit(X_train,Y_train)

# make prediction and print accuracy
prediction = model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))



'''


'\n# splitting into train and tests\nX_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size =.2, random_state=100)\n\n# make pipeline\nclf =  LogisticRegression()\ntext_clf = Pipeline([(\'tfidf\', TfidfVectorizer()),\n                     (\'clf\', LogisticRegression(solver=\'lbfgs\'))])\n\n# create and fit the model\n\nmodel.fit(X_train,Y_train)\n\n# make prediction and print accuracy\nprediction = model.predict(X_test)\nprint(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")\nprint(classification_report(Y_test, prediction))\n\n\n\n'

## applying the model on the data

In [212]:
data = pd.read_csv('june_data.csv')

In [213]:
data.shape

(100001, 8)

In [214]:
data['Datetime'] =  pd.to_datetime(data['Datetime'])
data['date']= data['Datetime'].dt.date

In [215]:
data.head()

,Unnamed: 0,Datetime,Tweet Id,Text,Username,loc,verified,media,date
0,0,2021-06-29 23:59:24+00:00,1410025136212881418,@ssnn911 لقاح كارونا اخذنه بس الباقي لازم,amira_9901,NaN,False,NaN,2021-06-29
1,1,2021-06-29 23:58:43+00:00,1410024964124708865,"لا ""PCR"" للمطعمين.. #سفارة_المملكة بالقاهرة تع...",bisha_R1,"بيشة, المملكة العربية السعودية",False,[Photo(previewUrl='https://pbs.twimg.com/media...,2021-06-29
2,2,2021-06-29 23:57:38+00:00,1410024694846312453,@j03dtmah @nmmesharii انا قلبي محروق اليوم اخذ...,moo30010,NaN,False,NaN,2021-06-29
3,3,2021-06-29 23:57:22+00:00,1410024624595902464,باتل الخير سوّى كل شي! بس باقي ينزّل لقاح حق ا...,ghdrooo,حبيبتي الكويت,False,NaN,2021-06-29
4,4,2021-06-29 23:57:14+00:00,1410024592803086340,شكوك حول فاعلية لقاح «#سينوفاك» بعد وفاة عدد م...,q8_news1,kuwait,False,NaN,2021-06-29


In [216]:
data = data[(data['media'].notna() == False) & (data['verified'] == False)]
data.drop(['Unnamed: 0', 'Datetime', 'verified', 'media'], axis= 1, inplace = True)

In [217]:
data.shape

(71369, 5)

In [218]:
# drop user_name
pattern = r'(@//[^"\s]+)|(@\w+)'
data['Text']= data.Text.str.replace(pattern, "")

In [219]:
# drop hashtag 
pattern = r'(#[A-Za-z0-9_]+)'
data['Text']= data.Text.str.replace(pattern, "")

In [220]:
# drop tweets that have links 
data = data[data['Text'] != data['Text'].str.contains('http')]

In [221]:
# drop emojy
data['Text'] = data['Text'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

In [222]:
# drop punctuation 
data['Text']= data['Text'].str.replace('[~,,)'',|,RT,/,(,\n,_,!,:,-,1,2,؟,3,4,5,6,7,8,9,0,&,;,#,?,.,="]', '')
data['Text']

0                    لقاح  كارونا اخذنه    بس الباقي  لازم
2          انا قلبي محروق اليوم اخذت لقاح عشان الجامعه ...
3        باتل الخير سوى كل شي بس باقي ينزل لقاح حق الكو...
4        شكوك حول فاعلية لقاح سينوفاك بعد وفاة عدد من م...
5        أتمنى يطلع لقاح ضد التفكير الزايدنبي نرقد هكي ...
                               ...                        
99990    يعني فيه اشياء مالها دخل فالتربيه هي من تركيبة...
99994     اعطوني موعدفي  حزيران لقاح فايزر في مستشفى صي...
99996     السلام عليكمانا سعودي اعمل بالكويتوأخذت لقاح ...
99997    بعض الكلام داء      وبعض الصمت دواء      والتج...
99998    -اليوم صار اسم استرازينكا مصدر لانعدام الثقة ع...
Name: Text, Length: 71369, dtype: object

In [223]:
data.head()

,Tweet Id,Text,Username,loc,date
0,1410025136212881418,لقاح كارونا اخذنه بس الباقي لازم,amira_9901,NaN,2021-06-29
2,1410024694846312453,انا قلبي محروق اليوم اخذت لقاح عشان الجامعه ...,moo30010,NaN,2021-06-29
3,1410024624595902464,باتل الخير سوى كل شي بس باقي ينزل لقاح حق الكو...,ghdrooo,حبيبتي الكويت,2021-06-29
4,1410024592803086340,شكوك حول فاعلية لقاح سينوفاك بعد وفاة عدد من م...,q8_news1,kuwait,2021-06-29
5,1410024222412394498,أتمنى يطلع لقاح ضد التفكير الزايدنبي نرقد هكي ...,mavic1215,في لُطف الله,2021-06-29


In [224]:
data['label']= text_clf.predict(data['Text'])

In [225]:
data.head()

,Tweet Id,Text,Username,loc,date,label
0,1410025136212881418,لقاح كارونا اخذنه بس الباقي لازم,amira_9901,NaN,2021-06-29,neu
2,1410024694846312453,انا قلبي محروق اليوم اخذت لقاح عشان الجامعه ...,moo30010,NaN,2021-06-29,neu
3,1410024624595902464,باتل الخير سوى كل شي بس باقي ينزل لقاح حق الكو...,ghdrooo,حبيبتي الكويت,2021-06-29,pos
4,1410024592803086340,شكوك حول فاعلية لقاح سينوفاك بعد وفاة عدد من م...,q8_news1,kuwait,2021-06-29,neu
5,1410024222412394498,أتمنى يطلع لقاح ضد التفكير الزايدنبي نرقد هكي ...,mavic1215,في لُطف الله,2021-06-29,neg


In [226]:
data['label'].value_counts()

neu    40067
neg    20544
pos    10758
Name: label, dtype: int64

In [227]:
data['loc'].value_counts()

المملكة العربية السعودية    2776
Kuwait                      1461
دولة الكويت                  778
Lebanon                      757
Kingdom of Saudi Arabia      712
                            ... 
Pc player, cats, anime         1
  ✨                            1
NEOM/JEDDAH                    1
Cleveland, OH                  1
محد مهتم                       1
Name: loc, Length: 9131, dtype: int64

In [228]:
ksa_t= data[data['loc'].isin(['المملكة العربية السعودية', 'Kingdom of Saudi Arabia'])]
ksa_t.reset_index()

,index,Tweet Id,Text,Username,loc,date,label
0,26,1410021008485064708,بدءا من ١ أغسطس ٢٠٢١هـسيشترط أخذ لقاح كورونا ل...,0education0,المملكة العربية السعودية,2021-06-29,neu
1,68,1410015684554366976,الي بيتصور الفتره هذي على انه اخذ الجرعة الثان...,khalled_gm,المملكة العربية السعودية,2021-06-29,pos
2,71,1410015396078534659,الحين اختبار القدرات لازم اخذ لقاح باشهر,zyad518FM,المملكة العربية السعودية,2021-06-29,neu
3,93,1410011874238812160,انا مقيم وحصلت على لقاح كورونا بمصر وعند تسجي...,sherif77552266,Kingdom of Saudi Arabia,2021-06-29,neu
4,95,1410011666570416129,إغلاق مواقع التلقيح الأمريكية في ولايات على ا...,Crypto_Arabs,المملكة العربية السعودية,2021-06-29,pos
...,...,...,...,...,...,...,...
3483,99856,1399645075941072896,تم اليوم بحمد لله أخذي الجرعة الثانية من لقاح ...,mandel_fq,المملكة العربية السعودية,2021-06-01,neu
3484,99878,1399642115542589442,افيدوني يرحمكم الله سألت كم مره مافي اجابةاخذ...,habibm102,المملكة العربية السعودية,2021-06-01,neu
3485,99880,1399641871241121796,١- مصيبه تخويف الناس من اخذ اللقاح الي هو الح...,nfhat43,المملكة العربية السعودية,2021-06-01,neg
3486,99956,1399633960393744384,صباح الخير والنور والسرور حمدلله على السلامهل...,omlean1234,المملكة العربية السعودية,2021-06-01,pos


In [229]:
ksa_t['Text'][68]

'الي بيتصور الفتره هذي على انه اخذ الجرعة الثانية من لقاح كورونا  بنعرف ان عمره فوق الخمسين عام الله يبارك في اعمارنا جميعا و يرفع عنا هذا الوباء '

In [230]:
ksa_t[ksa_t['label']== 'neg']['Text']

108       اخ بس لو فية لقاح ضد العاطفة نعطي عشاق مرابط ...
115      لاللتطعيمالاجباري كوررنا نفس الشي اعراض بعيده ...
120         كوررنا نفس الشي اعراض بعيده المدى حتى لو عد...
124       ياليت يتم تصنيعه في السعودية نبي لقاح نورة ضد...
248       صحيفة التايمز للإتحاد الأوربي  he European Un...
                               ...                        
98856     الدنيا سلف ودين والواحد يعرف الصح والغلطلكن ا...
99216     مركز طبية لاخذ لقاح كورونا في محافظة ينبع الي...
99319                   محد بيفضلك من حقك ترفض لقاح تجريبي
99331        لقاح كورونا الي الشفاء منه ٩٩% ما بناخذه و...
99880     ١- مصيبه تخويف الناس من اخذ اللقاح الي هو الح...
Name: Text, Length: 871, dtype: object

In [231]:
ksa_t['Text'][108]

' اخ بس لو فية لقاح ضد العاطفة نعطي عشاق مرابط وبيتروس الجار قال لنجم أدواردو أمسك الباب بس عشان أنخفض مستواة شيء بسيط بعد الاصابه مع العلم اني أشوفة لاعب مميز رغم أنخفاض مستواة مسكوة الباب ولا أحد قال شي عندهم ثقة بقرارات الأدارة وأن الأدارة مراح ترضى بلاعب مستواه غير جيد'

In [232]:
ksa_t['Text'][120]

'   كوررنا نفس الشي اعراض بعيده المدى حتى لو عدت بدون اعراض الفيروس يبقى في الجسم خامل يأثر على العقل والاعصاب واذا تكررت الاصابه اثرها اقوى كلها فينا لقاح ولا كورونا لا مفر الا بتوكل على الله'

In [233]:
ksa_t['Text'][99880]

' ١- مصيبه تخويف الناس من اخذ اللقاح الي هو الحل الوحيد بأذن الله لتجاوز هذا الوباالامر الاخر كونه لقاح طارئ هذا لأنه يجب أن يتم توزيعه بأسرع وقت نظرا لاجتياح الوباء بسرعه كبيره للكره الارضيه واللقاح اكتملت جميع مراحل تجربته واكمل سنه للتجارب قبل الاستخدام'